In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os 
# os.environ["ROOT"] = "E:\\"
from datetime import datetime, timedelta
try:
    from worker.song_history import load_df
    from worker.song_history import  get_durations
except:
    from song_history import load_df
    from song_history import  get_durations
import pandas as pd 
import plotly.offline as opy
import io, json, logging
import requests as r
import os 
import pickle

pd.options.plotting.backend = "plotly"

In [2]:
from worker.plot import artist_duration, select_month
from worker.authorize import get_token

In [4]:
df, mdf = load_df()

loaded from /home/pmhalvor/data/history.csv
Max played at 2022-09-09T19:08:16.224Z


In [5]:
from worker.song_history import * 
df = select_month(df, 7)

In [6]:
df.head()

,played_at,id,artist,name
27621,2022-02-11T13:04:16.389Z,68s6d9iCLRhviPqL6NWA5U,Totally Enormous Extinct Dinosaurs,Islas Canarias
27622,2022-02-11T13:05:39.359Z,6rr3OnsNcCHbnH9vvZLw6z,Wun Two,Daqui
27623,2022-02-11T13:10:05.301Z,0UAbqQRMrMVXR9lPcpWS7w,RUMTUM,Escape Cape
27624,2022-02-11T13:13:09.911Z,5BAXlRDW3YaPr3rfqV51TA,Instupendo,Comfort Chain
27625,2022-02-11T14:10:22.395Z,6N2yBySX0VhCVlVHKngirs,The Sweet Enoughs,Marshmallow


In [7]:
df.tail()

,played_at,id,artist,name
36515,2022-09-09T16:33:23.851Z,1M8t1j3Kv2qp97bdq5q4Vl,Tom Santa,Rainfall (Praise You)
36516,2022-09-09T16:35:41.889Z,20b1ZBMGOZjuGvNFncIIP7,Majestic,Time to Groove (feat. Nonô)
36517,2022-09-09T16:38:52.130Z,31B7wLv4yvtjDoTTmbnxeE,Fred again..,Jungle
36518,2022-09-09T16:41:34.211Z,7k7Huw9fxg3xer6ZNYyTMd,LF SYSTEM,It's Always Been There
36519,2022-09-09T19:08:16.224Z,2ICMnjBhI7xQhI6dV3Z78P,Jimmy Buffett,Brown Eyed Girl


In [8]:
durations = get_durations(list(set(df.id)))

658 new ids to check
Time difference: 920
Will be executing 14 API call(s)
Batch 0
Checking indexes: 0 -> 50
Empty DataFrame
Columns: [id, duration]
Index: []
Batch 1
Checking indexes: 50 -> 100
Empty DataFrame
Columns: [id, duration]
Index: []
Batch 2
Checking indexes: 100 -> 150
Empty DataFrame
Columns: [id, duration]
Index: []
Batch 3
Checking indexes: 150 -> 200
Empty DataFrame
Columns: [id, duration]
Index: []
Batch 4
Checking indexes: 200 -> 250
Empty DataFrame
Columns: [id, duration]
Index: []
Batch 5
Checking indexes: 250 -> 300
Empty DataFrame
Columns: [id, duration]
Index: []
Batch 6
Checking indexes: 300 -> 350
Empty DataFrame
Columns: [id, duration]
Index: []
Batch 7
Checking indexes: 350 -> 400
Empty DataFrame
Columns: [id, duration]
Index: []
Batch 8
Checking indexes: 400 -> 450
Empty DataFrame
Columns: [id, duration]
Index: []
Batch 9
Checking indexes: 450 -> 500
Empty DataFrame
Columns: [id, duration]
Index: []
Batch 10
Checking indexes: 500 -> 550
Empty DataFrame
Colum

In [9]:
# load durations pickle
pth = os.path.join(dir_path, 'store', 'duration_df.pkl')
with open(pth, 'rb') as f:
    durations = pickle.load(f)

In [11]:
len(durations)

658

In [18]:
id_list = list(set(df.id))
len(id_list)

4359

In [19]:
# check if current ids have already stored durations
for i in durations.id:
    try:
        idx = id_list.index(i)
        id_list.pop(idx)
    except:
        pass

len(id_list)

3701

In [20]:
# create new df 
new_durations = pd.DataFrame({
    'id':id_list,
    'duration': 0
})

In [21]:
ids = new_durations.id[new_durations.duration<1]

In [22]:
len(ids)

3701

In [24]:
token = get_token()

Time difference: 1587


In [26]:
batches = (len(id_list)//MAX_ID_COUNT) + 1
batches

75

In [27]:
URL = "https://api.spotify.com/v1/tracks"    # api-endpoint for recently played  
HEAD = {'Authorization': 'Bearer '+token}    # provide auth. crendtials

In [28]:
i = 0
print('Batch', i)
if i==(batches-1):
    batch_ids = ids[MAX_ID_COUNT*i:]  # last set of indices
    print("Checking indexes: {} -> {}".format(MAX_ID_COUNT*i, MAX_ID_COUNT*(i+1)) )
else:
    batch_ids = ids[MAX_ID_COUNT*i:MAX_ID_COUNT*(i+1)]  # forward indexing
    print("Checking indexes: {} -> {}".format(MAX_ID_COUNT*i, MAX_ID_COUNT*(i+1)) )

b_ids = ','.join(batch_ids)

Batch 0
Checking indexes: 0 -> 50


In [29]:
b_ids

'7mZASnos1MVmNVLHHg7nPJ,7zghUphvkHNEB7XvgxhOFG,71koc8oGlklnEFA2WHrv6K,0YnbUCbW8viWWikWXoVzB1,6O7jGud1gZ6s5uWDY3LJTb,0HFrCOmhCYXlv4NdEwRAuj,2h6fLwmfDafmP1ImJFFskj,6oCdAXZVWCaGfHlhRidpAM,6FalTkmo0IU0Vy7XbKjqp8,2EgB4n6XyBsuNUbuarr4eG,3dPQuX8Gs42Y7b454ybpMR,70D3I3hR1qmhqJ85omaXla,1MVBaNQjo36HZUvf3FiIgd,6N9q56oMzmCKVOUIJKCj3X,1FQif795tEHBFNSGOLq9Sz,4Yru811Q3TTIPK7acXOoHc,59yvfa8f14OzRbM11q7T8z,0MrI5860scv6Zhv8eJlLjY,32aCWjphTWNgtcXQrkvEsJ,6TgA4zU87WT2qt2YTG7SEo,4tTTW3DQboCekZGI8biPK1,1ASyxb9ktut4HcI74g7YPJ,08hYZJIjLZwUVDiMzzeu0q,7pnVHbcqbmP7LA1ey3ydvu,2ihCaVdNZmnHZWt0fvAM7B,07ABETRdek3ACMpRPvQuaT,0za13QFdMk58tq5cDwNzQx,1PVza5KmhSJhz12OewWciE,0hOe5KVXHijiKZaO2SR73J,01fBZdUaSQRqVrxpKzV2dj,4MQnXVR8DJvsySuXYuzUtf,3DWXwZRcFlwqu62xy4sASF,2hrFH10GDq8WyAxX9yQAyN,20EmHh9TVgaZUZ9UvmlvUv,3ImHLPyqf739WgzXKjPqfU,2y6dznvSGI38yR8149wqVS,1i7U0mv1bCD6uzXHfOjAQx,0BD9XqvXSSxlHsdBlLKhqA,1eCFz60zd7mAXgWLapPd9B,0mzU6qEuvtPrL9UcXAVKKU,2eaVMiKKrUAUy4Qy9UgM19,2UMWdssGd4rlT9oSeHUZh0,21FcbiXLqPC8GtlhgpyXFz,6yafxZlCDH

In [31]:
df.copy()

,played_at,id,artist,name
27621,2022-02-11T13:04:16.389Z,68s6d9iCLRhviPqL6NWA5U,Totally Enormous Extinct Dinosaurs,Islas Canarias
27622,2022-02-11T13:05:39.359Z,6rr3OnsNcCHbnH9vvZLw6z,Wun Two,Daqui
27623,2022-02-11T13:10:05.301Z,0UAbqQRMrMVXR9lPcpWS7w,RUMTUM,Escape Cape
27624,2022-02-11T13:13:09.911Z,5BAXlRDW3YaPr3rfqV51TA,Instupendo,Comfort Chain
27625,2022-02-11T14:10:22.395Z,6N2yBySX0VhCVlVHKngirs,The Sweet Enoughs,Marshmallow
...,...,...,...,...
36515,2022-09-09T16:33:23.851Z,1M8t1j3Kv2qp97bdq5q4Vl,Tom Santa,Rainfall (Praise You)
36516,2022-09-09T16:35:41.889Z,20b1ZBMGOZjuGvNFncIIP7,Majestic,Time to Groove (feat. Nonô)
36517,2022-09-09T16:38:52.130Z,31B7wLv4yvtjDoTTmbnxeE,Fred again..,Jungle
36518,2022-09-09T16:41:34.211Z,7k7Huw9fxg3xer6ZNYyTMd,LF SYSTEM,It's Always Been There


In [32]:
durations.head()

,id,duration
0,6UIWePGDVpRBlhdVixcqPe,131428.0
1,7hYezQFECAmU62Hm5Nm3rz,153028.0
2,2ZBBwFqj8TyEU9Skcj1TM0,430055.0
3,4CGEmTX3M5IzoLPS9HHMDq,145035.0
4,5Ld0nTNShgRHfMC7iDyeTb,189600.0


In [34]:
new_durations = pd.DataFrame({
    'id':id_list,
    # 'duration': 0
})

In [44]:
durations.id.dtype

dtype('O')

In [46]:
new_durations.id.dtype

dtype('O')

In [54]:
merged = durations.merge(new_durations, on="id", how="left")

In [55]:
merged[merged.duration.isna()]

,id,duration


# Update history 
History is out of sync, so the next few cells will force update these

In [11]:
token = get_token()

Time difference: 2255


In [14]:
from worker.song_history import get_recents

In [15]:
recents = get_recents(token)

In [19]:
df, mdf = load_df()
latest = df.tail(1)["played_at"].item()
print(latest)

2022-09-08T16:45:03.702Z


In [20]:
from worker.song_history import json_to_df 
recents_df = json_to_df(recents, latest)

New entry added: {'played_at': '2022-09-09T10:40:02.322Z', 'id': '4BGvTMV2DIwIa4ixT37fnG', 'artist': 'Ryan Griffin', 'name': 'Salt Lime & Tequila'}.
New entry added: {'played_at': '2022-09-09T10:43:02.402Z', 'id': '53DVsTZvB16BDP30XmfSef', 'artist': 'Watchhouse', 'name': 'Old Ties and Companions'}.
New entry added: {'played_at': '2022-09-09T10:45:49.535Z', 'id': '19MkwMrRjO7773zpuGss1o', 'artist': 'Tyler Childers', 'name': 'Shake the Frost (Live)'}.
New entry added: {'played_at': '2022-09-09T10:49:13.494Z', 'id': '3q3lSbfT5BnQlIvGwahgAD', 'artist': 'Luke Combs', 'name': 'The Great Divide'}.
New entry added: {'played_at': '2022-09-09T10:54:58.476Z', 'id': '3Ozx6IrGdoQyAworJzvBDE', 'artist': 'Colter Wall', 'name': 'Sleeping on the Blacktop'}.
New entry added: {'played_at': '2022-09-09T10:57:41.162Z', 'id': '2Dct3GykKZ58hpWRFfe2Qd', 'artist': 'Zach Bryan', 'name': 'Heading South'}.
New entry added: {'played_at': '2022-09-09T11:00:23.447Z', 'id': '4lb3LoC5Xa5FbL9uB0n3R9', 'artist': 'Whiske

In [21]:
recents_df

,id,played_at,artist,name
0,4BGvTMV2DIwIa4ixT37fnG,2022-09-09T10:40:02.322Z,Ryan Griffin,Salt Lime & Tequila
1,53DVsTZvB16BDP30XmfSef,2022-09-09T10:43:02.402Z,Watchhouse,Old Ties and Companions
2,19MkwMrRjO7773zpuGss1o,2022-09-09T10:45:49.535Z,Tyler Childers,Shake the Frost (Live)
3,3q3lSbfT5BnQlIvGwahgAD,2022-09-09T10:49:13.494Z,Luke Combs,The Great Divide
4,3Ozx6IrGdoQyAworJzvBDE,2022-09-09T10:54:58.476Z,Colter Wall,Sleeping on the Blacktop
5,2Dct3GykKZ58hpWRFfe2Qd,2022-09-09T10:57:41.162Z,Zach Bryan,Heading South
6,4lb3LoC5Xa5FbL9uB0n3R9,2022-09-09T11:00:23.447Z,Whiskey Myers,Bitch
7,0tIRswIxHkAXTF0cx2Eodb,2022-09-09T11:03:12.394Z,Vincent Neil Emerson,25 & Wastin' Time
8,2whwQ5AcH84MhNIXoAYKdi,2022-09-09T11:06:20.549Z,Watchhouse,Cavalry
9,0fCwTrRNfoNk962ati8n2K,2022-09-09T11:09:50.496Z,Tyler Childers,All Your'n


In [22]:
from worker.song_history import combine_dfs 

In [23]:
update_df = combine_dfs(df, recents_df)

In [24]:
from worker.song_history import df_to_csv

result = df_to_csv(update_df)

# Old debugging sandbox

In [4]:
df = df[df["played_at"] > (datetime.today() - timedelta(180)).strftime("%Y-%m-%dZ%H:%M:%S.000Z")]


In [5]:
df.head()

,played_at,id,artist,name
28338,2022-02-27T10:40:31.735Z,1No3OKWvdgtctLvSKQcREg,Khruangbin,Chocolate Hills
28339,2022-02-27T10:44:19.058Z,1zEkOtRGRBy85BU2dW0QsZ,Gavinco,Caravella - Edit
28340,2022-02-27T10:46:37.839Z,3EFJiOTKJ14jAWzuYxA46c,Remulak,Drifting
28341,2022-02-27T10:50:01.062Z,2FCQs4zlAjGeIh0KIqkEmD,The Olympians,Sagittarius By Moonlight
28342,2022-02-27T10:57:11.913Z,1hYLj6JNbs8RAwNNy3x3HG,Mark Kramer Trio,Mozart / Jazz Symphony in G Minor First Moveme...


In [6]:
len(df.id.unique())

3670

In [85]:
durations = get_durations(df.id.unique())

3670 new ids to check
Time difference: 2229
Will be executing 74 API call(s)


TypeError: sequence item 0: expected str instance, int found

In [24]:
durations.head()

,id,duration
0,1No3OKWvdgtctLvSKQcREg,0
1,1zEkOtRGRBy85BU2dW0QsZ,0
2,3EFJiOTKJ14jAWzuYxA46c,0
3,2FCQs4zlAjGeIh0KIqkEmD,0
4,1hYLj6JNbs8RAwNNy3x3HG,0


In [21]:
ids = durations.id[durations.duration<1]

In [22]:
ids

0       1No3OKWvdgtctLvSKQcREg
1       1zEkOtRGRBy85BU2dW0QsZ
2       3EFJiOTKJ14jAWzuYxA46c
3       2FCQs4zlAjGeIh0KIqkEmD
4       1hYLj6JNbs8RAwNNy3x3HG
                 ...          
3665    4bO6DljpuAeQh6HS20i0I5
3666    7cPojUq1xFAIcksvhFp4DV
3667    5Axapk6xMmSHK2A75EZiYo
3668    07ABETRdek3ACMpRPvQuaT
3669    4aeJDu3ftusgtKHpTxjcjk
Name: id, Length: 3670, dtype: object

In [34]:
MAX_ID_COUNT = 10
token = get_token()
batches = (len(ids)//MAX_ID_COUNT) + 1
print(f'Will be executing {batches} API call(s)')

Time difference: 715
Will be executing 368 API call(s)


In [35]:
URL = "https://api.spotify.com/v1/tracks"    # api-endpoint for recently played  
HEAD = {'Authorization': 'Bearer '+token}                       # provide auth. crendtials

In [36]:
durations.head(30)

,id,duration
0,1No3OKWvdgtctLvSKQcREg,0
1,1zEkOtRGRBy85BU2dW0QsZ,0
2,3EFJiOTKJ14jAWzuYxA46c,0
3,2FCQs4zlAjGeIh0KIqkEmD,0
4,1hYLj6JNbs8RAwNNy3x3HG,0
5,3yMYs3jJdAPxMgqb0CYZUG,0
6,6Rd2NhQnVeDBMUti7BiS90,0
7,5STMWcvgi6UiswwZyeWdTG,0
8,0upn2c2d9jGu6nI6hiTjW9,0
9,5YQxSoa9DJvbRz49X9BTc8,0


In [99]:
dir_path = os.environ["ROOT"]


for i in range(1):
    if i==(batches-1):
        batch_ids = ids[MAX_ID_COUNT*i:]  # last set of indices
    else:
        batch_ids = ids[MAX_ID_COUNT*i:MAX_ID_COUNT*(i+1)]  # forward indexing

    print(f"requesting {len(batch_ids)} ids")
    b_ids = ','.join(batch_ids)
    
    print(b_ids)
    PARAMS = {'ids':b_ids}	
    data = r.get(url=URL, headers=HEAD, params=PARAMS).json()

    batch_dur = []

    if data.get('tracks'):
        for track in data['tracks']:
            batch_dur.append(track['duration_ms'])

#             print(batch_dur)
#         durations.duration[batch_ids.index] = batch_dur
#     else:
#         print('No tracks in response')
#         print(data)

#     print(durations[durations.index.isin(batch_ids)])

requesting 10 ids
1No3OKWvdgtctLvSKQcREg,1zEkOtRGRBy85BU2dW0QsZ,3EFJiOTKJ14jAWzuYxA46c,2FCQs4zlAjGeIh0KIqkEmD,1hYLj6JNbs8RAwNNy3x3HG,3yMYs3jJdAPxMgqb0CYZUG,6Rd2NhQnVeDBMUti7BiS90,5STMWcvgi6UiswwZyeWdTG,0upn2c2d9jGu6nI6hiTjW9,5YQxSoa9DJvbRz49X9BTc8


In [89]:
batch_ids.index

Int64Index([10, 11, 12, 13, 14, 15, 16, 17, 18, 19], dtype='int64')

In [53]:
durations.duration[batch_ids.index]

20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
Name: duration, dtype: int64

In [100]:
batch_dur

[190433,
 234278,
 145774,
 210240,
 437880,
 380000,
 331392,
 197968,
 379229,
 146341]

In [101]:
dur = pd.DataFrame({"dur": batch_dur})

In [102]:
dur.index = batch_ids.index

In [103]:
dur

,dur
0,190433
1,234278
2,145774
3,210240
4,437880
5,380000
6,331392
7,197968
8,379229
9,146341


In [107]:
durations.duration[batch_ids.index] = batch_dur

C:\Users\perha\AppData\Local\Temp\ipykernel_22972\662607360.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [108]:
durations

,id,duration,dur
0,1No3OKWvdgtctLvSKQcREg,190433,NaN
1,1zEkOtRGRBy85BU2dW0QsZ,234278,NaN
2,3EFJiOTKJ14jAWzuYxA46c,145774,NaN
3,2FCQs4zlAjGeIh0KIqkEmD,210240,NaN
4,1hYLj6JNbs8RAwNNy3x3HG,437880,NaN
5,3yMYs3jJdAPxMgqb0CYZUG,380000,NaN
6,6Rd2NhQnVeDBMUti7BiS90,331392,NaN
7,5STMWcvgi6UiswwZyeWdTG,197968,NaN
8,0upn2c2d9jGu6nI6hiTjW9,379229,NaN
9,5YQxSoa9DJvbRz49X9BTc8,146341,NaN


In [59]:
durations.merge(batch_ids = batch_dur

C:\Users\perha\AppData\Local\Temp\ipykernel_22972\912534908.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [61]:
durations.head(30)

,id,duration
0,1No3OKWvdgtctLvSKQcREg,0
1,1zEkOtRGRBy85BU2dW0QsZ,0
2,3EFJiOTKJ14jAWzuYxA46c,0
3,2FCQs4zlAjGeIh0KIqkEmD,0
4,1hYLj6JNbs8RAwNNy3x3HG,0
5,3yMYs3jJdAPxMgqb0CYZUG,0
6,6Rd2NhQnVeDBMUti7BiS90,0
7,5STMWcvgi6UiswwZyeWdTG,0
8,0upn2c2d9jGu6nI6hiTjW9,0
9,5YQxSoa9DJvbRz49X9BTc8,0
